<a href="https://colab.research.google.com/github/Virtualexistence/Credit-Card-Approval-Automation/blob/main/CC_automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
import numpy as np
import pandas as pd
cc_apps = pd.read_csv("drive/MyDrive/Credit_Data/CC_data", header=None)
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


#Data Analysis

In [4]:
cc_apps.describe()

,2,7,10,14
count,690.000000,690.000000,690.00000,690.000000
mean,4.758725,2.223406,2.40000,1017.385507
std,4.978163,3.346513,4.86294,5210.102598
min,0.000000,0.000000,0.00000,0.000000
25%,1.000000,0.165000,0.00000,0.000000
50%,2.750000,1.000000,0.00000,5.000000
75%,7.207500,2.625000,3.00000,395.500000
max,28.000000,28.500000,67.00000,100000.000000


In [6]:
cc_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    object 
 13  13      690 non-null    object 
 14  14      690 non-null    int64  
 15  15      690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


#Train_Test Split

In [28]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(cc_apps, test_size=0.2, random_state=42) #To avoid test set contamination from train set we'll clean after the cleaning separately

#Data Cleaning

In [ ]:
train_set.replace("?",np.nan, inplace=True) #Replacing empty representation with NaN values
test_set.replace("?",np.nan, inplace=True)
#Object dtype columns have been replaced with NaN

In [30]:
train_set.fillna(train_set.mean(), inplace=True) #Imputation using mean values
test_set.fillna(test_set.mean(), inplace=True)
print(train_set.isna().sum())
print(test_set.isna().sum())

0     10
1      9
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13    13
14     0
15     0
dtype: int64
0     2
1     3
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
dtype: int64


<ipython-input-30-24376690de9a>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_set.fillna(train_set.mean(), inplace=True) #Imputation using mean values
<ipython-input-30-24376690de9a>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  test_set.fillna(test_set.mean(), inplace=True)


In [35]:
for col in cc_apps.columns:
  if cc_apps[col].dtypes == 'object':
    train_set.fillna(train_set[col].value_counts().index[0], inplace=True)
    test_set.fillna(test_set[col].value_counts().index[0],inplace=True)

In [37]:
print(train_set.isna().any().sum())
print(train_set.isna().any().sum()) #Check to see any existing NaN in the datesets

0
0


#Data Preprocessing

In [38]:
train_set = pd.get_dummies(train_set)
test_set = pd.get_dummies(test_set)

In [43]:
test_set = test_set.reindex(columns = train_set.columns, fill_value=0)

In [44]:
X_train, y_train = train_set.iloc[:,:-1], train_set.iloc[:,-1]
X_test, y_test = test_set.iloc[:,:-1], test_set.iloc[:,-1]

In [45]:
from sklearn.preprocessing import MinMaxScaler

In [46]:
scaler = MinMaxScaler(feature_range=(0,1))
re_X_train = scaler.fit_transform(X_train)
re_X_test = scaler.transform(X_test)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


#Building the Model

In [61]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [72]:
logreg = LogisticRegression()
t= [0.01, 0.001, 0.0001]
max_iter = [100,150,200]
param = dict(tol=t,max_iter=max_iter)

#Performance Evaluation

In [73]:
grid_model = GridSearchCV(estimator = logreg, param_grid=param,cv=5, n_jobs=-1)

In [75]:
results = grid_model.fit(re_X_train,y_train)

In [76]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [77]:
print("Obtained the best score of {} using the parameters {}".format(results.best_score_, results.best_params_))

Obtained the best score of 1.0 using the parameters {'max_iter': 100, 'tol': 0.01}


In [78]:
y_pred = grid_model.predict(re_X_test)
print("Accuracy of the logistic regression model: {:%} ".format(accuracy_score(y_test, y_pred)))

Accuracy of the logistic regression model: 100.000000% 


In [79]:
confusion_matrix(y_test, y_pred)

array([[70,  0],
       [ 0, 68]])